### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드
콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다


In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

# 0단계: 문서 로드
pdf_path = Path("data") / "콘텐츠분쟁해결_사례.pdf"
documents = loader.load()

# 1단계: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = text_splitter.split_documents(documents)

# 2단계: 임베딩 모델
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1536
)

# 3단계: 벡터스토어 생성
vectorstore = FAISS.from_documents(chunks, embeddings)

# 4단계: 검색기
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# 5단계: LLM 설정
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=2000
)

# 6단계: 프롬프트 템플릿
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# 7단계: QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 8단계: 분쟁 유형 분류 함수
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

# 9단계: 테스트 질문
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

# 10단계: 실행
for i, question in enumerate(test_questions, 1):
    dispute_type = classify_dispute_type(question)
    print(f"\n【테스트 {i}/5】")
    print(f"▶ 질문: {question}")
    print(f"▶ 분쟁 유형: {dispute_type}")
    
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]

    print(f"\n🧑‍⚖️ 답변:")
    print("-" * 50)
    print(answer)

    print(f"\n📄 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get("page", "N/A")
        preview = doc.page_content[:100].replace("\n", " ")
        print(f"  {j}. 페이지 {page}: {preview}...")

    print("-" * 50)



【테스트 1/5】
▶ 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
▶ 분쟁 유형: 게임

🧑‍⚖️ 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률적 조언을 드리겠습니다. 제시된 사례들을 참고하여 다음과 같이 조언드립니다.

1. **사례 분석**:
   - **사례 1 (2006)**: 시스템 오류로 아이템이 소멸되었으나, 계정 명의자가 아니어서 복구가 거부된 사례입니다. 계정 명의자가 복구 요청을 해야 한다는 점이 강조되었습니다.
   - **사례 23 (2009)**: 시스템 오류로 게임머니가 소실되었다고 주장했으나, 시스템 오류가 확인되지 않아 복구가 거부된 사례입니다. 시스템 오류가 발생했음을 입증하는 것이 중요합니다.

2. **관련 법령 및 조항**:
   - 게임사의 약관 및 운영정책이 중요합니다. 약관에 시스템 오류로 인한 복구 절차가 명시되어 있는지 확인해야 합니다.
   - 민법 제250조 및 제251조는 도난 및 유실물에 관한 규정을 다루고 있으나, 시스템 오류와 직접적인 관련은 없습니다.

3. **실무적 해결방안**:
   - **증거 수집**: 시스템 오류가 발생했음을 입증할 수 있는 증거를 수집하세요. 예를 들어, 오류 발생 시각의 스크린샷, 다른 사용자들의 유사한 불만 사례 등을 확보하세요.
   - **계정 명의 확인**: 계정 명의자가 본인이 맞는지 확인하고, 명의가 다르다면 명의자에게 복구 요청을 의뢰하세요.
   - **게임사와의 협의**: 게임사의 고객센터에 시스템 오류로 인한 피해를 설명하고, 복구 요청을 공식적으로 제출하세요. 이때, 수집한 증거를 함께 제출하는 것이 좋습니다.
   - **소비자 보호 기관에 문의**: 한국소비자원 등 소비자 보호 기관에 상담을 요청하여 추가적인 조언을 받